In [1]:
import numpy as np 
import gymnasium as gym
import ale_py
import pickle

gym.register_envs(ale_py)

env = gym.make('PongNoFrameskip-v4')
observation, info = env.reset()
# obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
print(observation.shape)

(210, 160, 3)


A.L.E: Arcade Learning Environment (version 0.10.1+6a7e0ae)
[Powered by Stella]


In [2]:
def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0  # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float64).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations


In [3]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def relu(vector):
    vector[vector < 0] = 0
    return vector

def policy_forward(observation_matrix, weights):
    """ Based on the observation_matrix and weights, compute the new hidden layer values and the new output layer values (probability of taking action 2)"""
    hidden_layer_values = np.dot(weights['1'], observation_matrix)
    hidden_layer_values = relu(hidden_layer_values)
    output_layer_values = np.dot(weights['2'], hidden_layer_values)
    output_layer_values = sigmoid(output_layer_values)
    return hidden_layer_values, output_layer_values

In [4]:
def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

In [5]:
def discount_rewards(rewards, gamma):
    """ Actions you took x steps before the end result are less important to the overall result than an action you took a step ago.
    This implements that logic by discounting the reward on previous actions based on how long ago they were taken"""
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(range(0, rewards.size)):
        if rewards[t] != 0:
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

def discount_with_rewards(gradient_log_p, episode_rewards, gamma):
    """ discount the gradient with the normalized rewards """
    discounted_episode_rewards = discount_rewards(episode_rewards, gamma)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_episode_rewards -= np.mean(discounted_episode_rewards)
    discounted_episode_rewards /= np.std(discounted_episode_rewards)
    return gradient_log_p * discounted_episode_rewards

<img src="./images/derivative.png" width="auto"  />

In [6]:
def policy_backward(gradient_log_p, hidden_layer_values, observation_values, weights):
    """ See here: http://neuralnetworksanddeeplearning.com/chap2.html"""
    delta_L = gradient_log_p # (10, 1) I say 10 just random example
    dC_dw2 = np.dot(hidden_layer_values.T, delta_L).ravel() # ∂L/∂w2 = ∂L/∂f * ∂f/∂w2 where last one is just hidden layer values (10,200,1).T = (1,200,10) dot (10,1) = (1,200,1).ravel = 200
    delta_l2 = np.outer(delta_L, weights['2']) # ∂L/∂x2 = ∂L/∂f * ∂L/∂x2 where last one is weights of w2 (10,1)outer(200) = (10,200)
    delta_l2 = relu(delta_l2) # ∂L/∂a don't forget that we had acticavation function, so we need to get rid of negative numbers before back propagate further (10,200)
    dC_dw1 = np.dot(delta_l2.T, observation_values) # ∂L/∂w1 = ∂L/∂a * ∂a/∂w1  (10,200).T * (10,1600) => (200,10)*(10,1600) = (200,1600)
    return {
        '1': dC_dw1,
        '2': dC_dw2
    }

<img src="./images/derivative_2.png" width="auto"  />

In [7]:
def update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate):
    """ See here: http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop"""
    epsilon = 1e-5
    for layer_name in weights.keys():
        g = g_dict[layer_name]
        expectation_g_squared[layer_name] = decay_rate * expectation_g_squared[layer_name] + (1 - decay_rate) * g**2
        weights[layer_name] += (learning_rate * g)/(np.sqrt(expectation_g_squared[layer_name] + epsilon))
        g_dict[layer_name] = np.zeros_like(weights[layer_name]) # reset batch gradient buffer

In [8]:
 # hyperparameters
batch_size = 10 # every how many episodes to do a param update?
gamma = 0.99 # discount factor for reward
decay_rate = 0.99  # decay factor for RMSProp leaky sum of grad^2
num_hidden_layer_neurons = 200
input_dimensions = 80 * 80
learning_rate = 1e-4
render = False

In [ ]:
episode_number = 0
reward_sum = 0
running_reward = None
prev_processed_observations = None
resume = False


if resume:
    weights = pickle.load(open('./models/save.p', 'rb'))
else:
    weights = {
        '1': np.random.randn(num_hidden_layer_neurons, input_dimensions) / np.sqrt(input_dimensions),
        '2': np.random.randn(num_hidden_layer_neurons) / np.sqrt(num_hidden_layer_neurons)
    }


In [10]:
 # To be used with rmsprop algorithm (http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop)
expectation_g_squared = {}
g_dict = {}
for layer_name in weights.keys():
    expectation_g_squared[layer_name] = np.zeros_like(weights[layer_name])
    g_dict[layer_name] = np.zeros_like(weights[layer_name])

In [ ]:
episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], []

while True:
    if render: env.render()

    processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)

    hidden_layer_values, up_probability = policy_forward(processed_observations, weights)
    episode_observations.append(processed_observations)
    episode_hidden_layer_values.append(hidden_layer_values)

    action = choose_action(up_probability) # toss the coin, up or down 
    # carry out the chosen action
    observation, reward, terminated, truncated, info = env.step(action)

    reward_sum += reward
    episode_rewards.append(reward)

    # see here: http://cs231n.github.io/neural-networks-2/#losses
    # ∂L/∂f = predicted_label(0 or 1) - true_label(0 or 1) or (sigmoid(f) - y)
    # take negative of gradient to reduce error  when its added in future or -∂L/∂f = y - sigmoid(f)
    fake_label = 1 if action == 2 else 0
    loss_function_gradient = fake_label - up_probability # grad that encourages the action that was taken to be taken 
    episode_gradient_log_ps.append(loss_function_gradient)

    if truncated or terminated:

        episode_number += 1
        
        # Combine the following values for the episode
        episode_hidden_layer_values = np.vstack(episode_hidden_layer_values)
        episode_observations = np.vstack(episode_observations)
        episode_gradient_log_ps = np.vstack(episode_gradient_log_ps)
        episode_rewards = np.vstack(episode_rewards)

        # Tweak the gradient of the log_ps based on the discounted rewards
        # modulate the gradient with advantage (PG magic happens right here.)
        # so each gradient loss * discounted rewards hmmmm
        episode_gradient_log_ps_discounted = discount_with_rewards(episode_gradient_log_ps, episode_rewards, gamma)
        gradient = policy_backward(
            episode_gradient_log_ps_discounted,
            episode_hidden_layer_values,
            episode_observations,
            weights
        )

        # Sum the gradient for use when we hit the batch size ( accumulate grad over batch)
        for layer_name in gradient:
            g_dict[layer_name] += gradient[layer_name]
        
        if episode_number % batch_size == 0:
            update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate)

        episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], [] # reset values
        observation, info = env.reset() # reset env
        
        # boring book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print('\r %f resetting env. episode reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
        if episode_number % 100 == 0: pickle.dump(weights, open('./models/save.p', 'wb'))
        reward_sum = 0
        prev_processed_observations = None
       



